In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
from time import time
import random
import turtle
import msvcrt

In [2]:
class Player:
    name = ""
    character = ""
    winner = ""

In [3]:
board_arr = np.array([[" ", " ", " ", " ", " "],
                      [" ", " ", " ", " ", " "],
                      [" ", " ", " ", " ", " "],
                      [" ", " ", " ", " ", " "],
                      [" ", " ", " ", " ", " "]])

In [4]:
class TeekoGame:
    cant = 0
    state = False
    def moveBoads():
        for row in 5:
            for column in 5:
                if board_arr[row][column] == " ":
                    cant = cant + 1
        if cant > 0:
            state = True
        return state

    def validMove(pos):
        state = False
        if board_arr[pos][pos]==" ":
            state = True
        return state

    def newMove(pos, state):
        print("Choose anothe possition")
        if state == True:
            newPos = msvcrt.getch()
        else:
            newPos = pos
        return newPos

    def playerTurn(pos, piece, move, aux):
        if TeekoGame.validMove(pos):
            board_arr[pos][pos] = piece
        else:
            newPos = TeekoGame.newMove(aux, move)
            while (TeekoGame.validMove(newPos)==False):
                newPos=TeekoGame.newMove(aux, move)
            board_arr[newPos][newPos] = piece
    
    def isWinner(pos, piece, cont):
        if (board_arr[pos][0] == piece and board_arr[pos][1] == piece and board_arr[pos][2] == piece and board_arr[pos][3] == piece):
            cont=cont+1
        if (board_arr[0][pos] == piece and board_arr[1][pos] == piece and board_arr[2][pos] == piece and board_arr[3][pos] == piece):
            cont=cont+1
        if (board_arr[0][0] == piece and board_arr[1][1] == piece and board_arr[2][2] == piece and board_arr[3][3] == piece):
            cont=cont+1
        if (board_arr[0][2] == piece and board_arr[1][1] == piece and board_arr[2][0] == piece):
            cont=cont+1
        return cont

    def Winner(pos, piece, cont):
        state = False
        cont = 0
        for i in 5:
            cont = TeekoGame.isWinner(pos, piece, cont)
        if cont > 0:
            state = True
        return state

In [6]:
class PlayGame:

    def movePlayer(board, player):
        print("Your turn player")
        pos = msvcrt.getch()
        board.playerTurn(pos, player.piece, True, aux)
        board.GameBoard()
    
    def InitGame(player1, player2):
        teekoGame = TeekoGame
        player = Player
        while teekoGame.moveBoads() and not(teekoGame.Winner(player1.character)) and not(teekoGame.Winner(player2.character)):
            PlayGame.movePlayer(teekoGame, player1)
            if teekoGame.moveBoads() and not(teekoGame.Winner(player1.character)) and not(teekoGame.Winner(player1.character)):
                PlayGame.movePlayer(teekoGame, player2)
        if teekoGame.Winner(player1.character):
            player1.winner = True
            player = player1
        else:
            if teekoGame.Winner(player2.character):
                player2.winner = True
                player = player2
        return player   

In [7]:
class Main:

    playGame = PlayGame

    player1 = Player
    player2 = Player

    print("Insert name player 1")
    name1 = msvcrt.getch()
    print("Insert name player 2")
    name2 = msvcrt.getch()

    playGame.InitGame(player1, name1)
    playGame.InitGame(player2, name2)
    player = playGame.InitGame(player1, player2)

    if player.winner:
        print(player.name + "Wins!")
    else:
        print(player.name + "Draw!")


Insert name player 1
